In [60]:
import numpy as np
from deap import base, creator, tools
import random

# 初始化三维矩阵
X = np.zeros((100, 50, 50))

# 定义选择的数量
N = 5000

# 生成拉丁方
def generate_latin_square(n):
    latin_square = np.zeros((n, n), dtype=int)
    for i in range(n):
        latin_square[i] = (np.arange(n) + i) % n
    return latin_square

# 定义适应度函数
def fitness_function(ind):
    # 提取选择的索引
    i_ind = ind[:N]
    j_ind = ind[N:2*N]
    k_ind = ind[2*N:3*N]
    
    # 计算频数
    freq_i = np.bincount(i_ind, minlength=100)
    freq_j = np.bincount(j_ind, minlength=50)
    freq_k = np.bincount(k_ind, minlength=50)

    # 合并频数
    combined_freq = np.concatenate((freq_i, freq_j, freq_k))
    
    # 返回最小频数（作为适应度）
    return np.min(combined_freq),

# 遗传算法参数设置
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # 最大化
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# 使用拉丁方初始化个体
latin_square_i = generate_latin_square(100)  # 生成100x100的拉丁方用于i
latin_square_jk = generate_latin_square(50)   # 生成50x50的拉丁方用于j和k

toolbox.register("i_indices", random.choices, (lambda: latin_square_i[random.randint(0, 99)].tolist())(), k=N)  # i 的选择
toolbox.register("j_indices", random.choices, (lambda: latin_square_i[random.randint(0, 49)].tolist())(), k=N)  # j 的选择
toolbox.register("k_indices", random.choices, (lambda: latin_square_i[random.randint(0, 49)].tolist())(), k=N)  # k 的选择

toolbox.register("individual", tools.initIterate, creator.Individual, 
                 lambda: toolbox.i_indices() + toolbox.j_indices() + toolbox.k_indices())


toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 遗传算法执行
population = toolbox.population(n=500)  # 种群大小
ngen = 200  # 最大迭代次数

for gen in range(ngen):
    # 选择
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 交叉和变异
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:  # 交叉概率
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.3:  # 变异概率
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 评估新一代个体
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # 替换种群
    population[:] = offspring

# 输出最佳解
fits = [ind.fitness.values[0] for ind in population]
best_idx = np.argmax(fits)
best_ind = population[best_idx]

# 计算最佳解的最小频数
best_freq_i = np.bincount(best_ind[:N], minlength=100)
best_freq_j = np.bincount(best_ind[N:2*N], minlength=50)
best_freq_k = np.bincount(best_ind[2*N:3*N], minlength=50)
combined_best_freq = np.concatenate((best_freq_i, best_freq_j, best_freq_k))
min_freq = np.min(combined_best_freq)

print("最佳最小频数:", combined_best_freq,min_freq)


最佳最小频数: [40 51 59 48 39 50 52 49 58 56 46 59 48 59 49 59 49 46 42 50 55 44 57 57
 48 58 49 59 51 49 42 50 47 51 47 47 53 49 55 51 54 59 49 59 55 45 44 46
 47 46 44 42 40 52 54 45 46 46 50 46 47 43 42 49 48 54 58 54 43 47 39 47
 50 49 44 54 43 49 56 62 57 61 43 40 57 40 40 51 54 61 44 57 55 57 48 58
 48 60 44 50 43 48 44 55 57 55 48 53 44 59 48 48 54 48 49 57 44 53 51 39
 47 52 42 56 51 47 45 50 59 45 45 39 44 49 58 42 53 47 44 55 42 41 52 58
 45 45 46 41 55 55 43 52 47 43 51 71 42 57 44 56 49 50 49 62 54 50 41 50
 42 58 54 52 47 47 53 55 56 56 49 49 47 45 61 53 54 45 49 51 49 48 60 55
 40 53 62 55 52 56 43 46 54 41 48 58 45 55 44 40 53 50 47 54 40 58 42 69
 39 44 40 47 45 48 45 57 47 63 55 55 45 43 48 57 53 43 44 57 46 50 49 54
 42 47 48 44 50 43 44 41 65 44 49 47 56 53 51 62 54 52 50 48 46 53 51 62
 55 53 44 42 52 61 51 40 52 47 49 49 63 49 63 55 50 57 45 66 55 54 51 48
 49 53 39 45 52 49 54 43 48 42 55 46] 39


In [42]:
def generate_latin_square(n):
    latin_square = np.zeros((n, n), dtype=int)
    for i in range(n):
        latin_square[i] = (np.arange(n) + i) % n
    return latin_square
latin_square_i = generate_latin_square(100) 

# latin_square_i[random.randint(0, 99)].tolist()
arr = lambda: latin_square_i[random.randint(0, 99)].tolist()
print(arr())

[77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]


In [50]:
random.choices(range(100),k=5)

[0, 90, 77, 64, 51]